# Documentation

This analysis helps to estimate the quality of the stationary glide model by calculating the resultant force at the wing for every datapoint, comparing it to the expected force and returning both a mean force deviation and a deviation percentage.

Inputs:
- INPUT_DIRECTORY: path to the directory containing .igc files
- OUTPUT_DIRECTORY: output data will be exported to this directory
- FILE_EXTENSION: set to `.igc`

# Imports

In [ ]:
import os
import sys
import pandas as pd
from datetime import datetime
from typing import List, Tuple

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

import constants as constants
import helpers.file_processor as file_processor
import helpers.data_visualizer as data_visualizer
import algorithms.speed_analyzer as speed_analyzer
import helpers.quality_analyzer as quality_analyzer
import algorithms.c_values_analyzer as c_values_analyzer
import algorithms.pressure_analyzer as pressure_analyzer


# Variables

In [ ]:
INPUT_DIRECTORY: str = "INPUT_DIRECTORY/" # end with "/
OUTPUT_DIRECTORY: str = "OUTPUT_DIRECTORY/" # end with /
FILE_EXTENSION: str = ".igc"

# Initialisation

In [ ]:
file_processor = file_processor.FileProcessor()
speed_analyzer = speed_analyzer.SpeedAnalyzer()
data_visualizer = data_visualizer.DataVisualizer()
c_analyzer = c_values_analyzer.CAnalyzer()
p_analyzer = pressure_analyzer.PressureAnalyzer()
quality_analyzer = quality_analyzer.QualityAnalyzer()

theoretical_reference: pd.DataFrame = pd.read_csv(f"{constants.THEORETICAL_REFERENCE_PATH}")
original_reference: pd.DataFrame = pd.read_csv(f"{constants.ORIGINAL_REFERENCE_PATH}")

timestamp: str = datetime.now().strftime("%Y%m%d-%H%M%S")

# File Listing

In [ ]:
file_paths: List[str] = file_processor.get_file_paths(path=INPUT_DIRECTORY, file_extension=FILE_EXTENSION)

print(f"Importing files for the following conditions:")
print(f"--> Directory: {INPUT_DIRECTORY}")
print(f"--> File extension: {FILE_EXTENSION}")
print(f"--> Theoretical polar: {constants.THEORETICAL_REFERENCE_PATH.split("/")[-1]}")
print(f"--> Original reference: {constants.ORIGINAL_REFERENCE_PATH.split("/")[-1]}")
print()
print("Files:")
print(f"--> Found {len(file_paths)} files.")
print(f"--> The processing is initiated.")

# File Processing

In [ ]:
data_raw: pd.DataFrame = speed_analyzer.process_raw_data(file_paths=file_paths)

# Data Preprocessing

In [ ]:
# original data
original_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=original_reference, reference=True)
airspeed_original_data: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=original_reference_filtered))
c_values_original_simplified: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_original_data, algorithm=False)

# experimental data
data_raw_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=data_raw)
smoothed_data_raw: pd.DataFrame = speed_analyzer.savgol_filter(data=data_raw_filtered)
smoothed_data_grouped: pd.DataFrame = speed_analyzer.group_data(data=smoothed_data_raw)
airspeed_smoothed_data_grouped: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=smoothed_data_grouped))
c_values_experimental_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_smoothed_data_grouped, algorithm=True)

# Quality Analysis

In [ ]:
print(f"Quality analysis:")
print(f"--> The reference calculations are conducted for all datapoints of the experimental dataset.")

quality_index_experimental: Tuple[float, float] = quality_analyzer.analyze_quality(data=c_values_experimental_optimized, model=True)

print(f"----> The mean deviation of the expected resulting force is {abs(round(quality_index_experimental[0], 2))} N.")
print(f"----> The mean deviation percentage of the expected resulting force is {abs(round(quality_index_experimental[1], 2))} %.")

print(f"--> The reference calculations are conducted for all datapoints of the theoretical dataset using the approximation model of the original paper (p. 40, fig. 56)")

quality_index_original: Tuple[float, float] = quality_analyzer.analyze_quality(data=c_values_original_simplified, model=False)

print(f"----> The mean deviation of the expected resulting force is {abs(round(quality_index_original[0], 2))} N.")
print(f"----> The mean deviation percentage of the expected resulting force is {abs(round(quality_index_original[1], 2))} %.")

# Quality Visualisation

In [ ]:
data_visualizer.visualize_quality_deviation(deviation_data=quality_index_experimental[2], title="Qualitätsabweichung basierend auf angenäherten Polaren (optimierte Daten, 31.03.2024)")
data_visualizer.visualize_quality_deviation(deviation_data=quality_index_original[2], title="Qualitätsabweichung basierend auf angenäherten Polaren (originale Daten, 24.10.2024)")

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")